In [1]:
import networkx as nx
import numpy as np
import scipy.stats
import pickle
%matplotlib inline

In [2]:
def KLD_entropy(graph, characteristic, characteristic_random_logprob):
    degree_total_g = sum(graph.degree().values())
    n = len(graph.nodes())
    p = float(degree_total_g) / float(n) / float(n-1)
    #random_graph = nx.erdos_renyi_graph(n, p)
    a = characteristic(graph)
    b = characteristic_random_logprob(n, p)
    a_non_zero = a[np.nonzero(a)]
    S = 0
    S += np.sum(np.multiply(a_non_zero, np.log(a_non_zero)))
    S += np.sum(np.multiply(-a, b))
    return S

In [19]:
def KLD_entropy_laplacian(graph, characteristic, characteristic_random_log):
    degree_total_g = sum(graph.degree().values())
    n = len(graph.nodes())
    p = float(degree_total_g) / float(n) / float(n-1)
    random_graph = nx.erdos_renyi_graph(n, p)
    a = characteristic(graph)
    b = characteristic_random_log(random_graph)
    a_non_zero = a[np.nonzero(a)]
    S = 0
    S += np.sum(np.multiply(a_non_zero, np.log(a_non_zero)))
    S += np.sum(np.multiply(-a, b))
    return S

In [4]:
def degree_distribution(graph):
    a = np.bincount(graph.degree().values())
    a.resize(len(graph.nodes()))
    a = a.astype(np.float)
    a = np.divide(a, np.sum(a))
    return a

In [5]:
def degree_distribution_random_logprob(n, p):
    pdf = []
    for i in xrange(n):
        pdf.append(scipy.stats.binom.logpmf(i, n-1, p))
        
    pdf = np.array(pdf)
    
    pdf[pdf == -np.inf] = np.iinfo(np.int).min
    
    return pdf

In [32]:
n = 2000
K_5 = nx.complete_graph(n)
maze=nx.sedgewick_maze_graph()
k_reg = nx.random_regular_graph(n/2, n)
bull = nx.bull_graph()
small_world = nx.newman_watts_strogatz_graph(n, 2, 0.1)

In [7]:
rg = nx.erdos_renyi_graph(n, 0.1)

In [16]:
KLD_entropy(small_world, degree_distribution, degree_distribution_random_logprob)

0.55557896989140321

In [11]:
def laplacian_spectrum_log(graph):
    return np.log(laplacian_spectrum(graph))

In [13]:
def laplacian_spectrum(graph):
    a = nx.laplacian_spectrum(graph)
    a /= sum(a)
    a[a <= 0] = np.finfo(np.float).tiny
    return a

In [15]:
def adjacency_spectrum_log(graph):
    return np.log(adjacency_spectrum(graph))

In [17]:
def adjacency_spectrum(graph):
    a = nx.adjacency_spectrum(graph)
    a /= sum(a)
    a[a <= 0] = np.finfo(np.float).tiny

In [33]:
entropies = []
for i in range(10):
    entropies.append(KLD_entropy_laplacian(small_world, laplacian_spectrum, laplacian_spectrum_log))

In [34]:
np.mean(entropies)

1.2355298355126763

In [35]:
np.std(entropies)

0.21417269955406329

In [ ]:
KLD_entropy_laplacian(small_world, adjacency_spectrum, adjacency_spectrum_log)